In [1]:
import os
import matplotlib.pyplot as plt
from helpers import download_file, read_json_file, download_json_geodata_from_item, calculate_area_from_file, smooth_data
directory_path = 'data'

## Download files

In [ ]:
# Download root file
download_url = 'https://deepstatemap.live/api/history/public'
root_json_file_name = 'data/public.json'

if not os.path.exists(root_json_file_name):
    download_file(download_url, root_json_file_name)

# Download days files
count_files_to_download_from_today = 20

global_json_data = read_json_file(root_json_file_name)
global_json_data = reversed(global_json_data)
global_json_data = list(global_json_data)[:count_files_to_download_from_today]

for json_data in global_json_data:
    download_json_geodata_from_item(json_data, sleep_seconds = 1)
    print('Downloaded file', json_data['createdAt'])

print('Done')

## Build chart

In [ ]:

# Data analysis of last 280 days
count_files_to_analyze_from_today = 280

json_files = [f for f in os.listdir(directory_path) if f.endswith(".json") and 'public' not in f]
json_files = list(sorted(json_files))
json_files = json_files[-count_files_to_analyze_from_today:]

result = {}

for json_file_name in sorted(json_files):
    date_str = json_file_name.split('.')[0]
    total_area_km2 = calculate_area_from_file(f'data/{json_file_name}')
    result[date_str] = total_area_km2

window_size = 7
deltas, dates = smooth_data(result, window_size)

deltas = deltas[window_size:]
dates = dates[window_size:]

# Chart
plt.figure(figsize=(10, 6))
plt.plot(dates, deltas)

plt.xticks(rotation=45)
plt.legend()

plt.xticks(dates[::30], rotation=45)
plt.grid(axis='x', which='both', linestyle='-', linewidth=0.5)
plt.grid(axis='y', which='both', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()